In [1]:
import matplotlib.pyplot as plt
import numpy
import pandas

# See https://keras.io/
# for extennsive documentation
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense

import sklearn.utils

import pickle
def fnSaveObject(save_object, sFileName):
    with open(sFileName, "wb") as file:
        pickle.dump(save_object, file)

def fnLoadObject(sFileName):
    with open(sFileName, 'rb') as file:
        load_object = pickle.load(file)
    return load_object

# sans_models = ['core_shell_cylinder', 'core_shell_ellipsoid', 'three_pearl_necklace', 'lamellar']
sans_models = ['core_shell_cylinder', 'core_shell_cylinder', 'core_shell_cylinder', 'core_shell_cylinder']

2023-03-23 21:14:39.648429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# data preparation
def y_encode(model, ydata, modellist, ydim):
    def onehot(y_cl, num_classes):
        y = numpy.zeros((len(y_cl), num_classes))
        y[numpy.arange(len(y_cl)), y_cl] = 1
        return y

    result = []
    class_number = None
    for i, modelname in enumerate(modellist):
        # regression outputs
        if model == modelname:
            # fill output for particular model with training values
            result.append(ydata)
            class_number = i
        else:
            # the outputs for the other models are set to zero
            result.append(numpy.zeros(shape=(ydata.shape[0], ydim[i])))

    # classification output, one-hot encoding for the particular model
    oh = numpy.full((ydata.shape[0]), class_number)
    oh = onehot(oh, len(modellist))
    result.append(oh)

    return result


# step-by-step model loading and data augmentation
data_x_pandas = []
data_y_pandas = []
data_x = None
data_y = None
ydim = []
for sansmodel in sans_models:
    print(sansmodel)
    data_x_pandas.append(fnLoadObject('train_'+sansmodel+'_x.dat'))
    data_y_pandas.append(fnLoadObject('train_'+sansmodel+'_y.dat'))
    # number of parameters knowing that scale and sld_solvent will be dropped
    ydim.append(data_y_pandas[-1][0].shape[0]-2)


for i, sansmodel in enumerate(sans_models):
    parlist = data_y_pandas[i][0]['par'].values.tolist()
    modparlist = []
    for par in parlist:
        if 'sld' in par and par != 'sld_solvent':
            modparlist.append(par)

    for frame in data_y_pandas[i]:
        solvent_sld = frame.loc[frame.par =='sld_solvent', 'value'].squeeze()

        # reduce all sld values in the model to their difference to the solvent sld
        for par in modparlist:
            sld = frame.loc[frame.par == par, 'value'].squeeze()
            frame.loc[frame.par == par, 'value'] = solvent_sld - sld

        # remove solvent sld and scale factor, since they cannot be resolved
        index_names = frame[ frame['par'] == 'scale'].index
        frame.drop(index_names, inplace = True)
        index_names = frame[ frame['par'] == 'sld_solvent'].index
        frame.drop(index_names, inplace = True)

    new_x = numpy.row_stack([frame['I'].to_numpy() for frame in data_x_pandas[i]]).astype('float32')
    new_x = numpy.log10(new_x)
    new_y = numpy.row_stack([frame['value'].to_numpy() for frame in data_y_pandas[i]]).astype('float32')
    new_y = y_encode(sansmodel, new_y, sans_models, ydim)

    if data_x is None:
        data_x = new_x
    else:
        data_x = numpy.concatenate((data_x, new_x), axis=0)

    if data_y is None:
        data_y = new_y
    else:
        for j in range(len(sans_models)+1):
            data_y[j] = numpy.concatenate((data_y[j], new_y[j]), axis=0)

core_shell_cylinder
core_shell_cylinder
core_shell_cylinder
core_shell_cylinder


In [ ]:
# model construction
input_dim = data_x.shape[1]
n_input_layers = 4
n_branch_layers = 4
output_layers = []

node_n = input_dim

activation = "relu"
reg_strategy = keras.regularizers.l1_l2(l1=0.001, l2=0.001)  # use L1 and L2 regularization

# common input layers
sans_input = keras.Input(shape=(input_dim, ))
x = layers.Dense(node_n, activation=activation, kernel_regularizer=reg_strategy)(sans_input)
for _ in range(n_input_layers-1):
    x = layers.Dense(node_n, activation=activation, kernel_regularizer=reg_strategy)(x)

# split into different model regression layers plus classification
# regression
for j, sans_model in enumerate(sans_models):
    x2 = layers.Dense(node_n, activation=activation, kernel_regularizer=reg_strategy)(x)
    for i in range(n_branch_layers-1):
        x2 = layers.Dense(node_n, activation=activation, kernel_regularizer=reg_strategy)(x2)
    output_layers.append(Dense(data_y[j].shape[1], activation="linear")(x2))

# classification
x2 = layers.Dense(node_n, activation=activation, kernel_regularizer=reg_strategy)(x)
for _ in range(n_branch_layers-1):
    x2 = layers.Dense(node_n, activation=activation, kernel_regularizer=reg_strategy)(x2)
output_layers.append(Dense(len(sans_models), activation="softmax")(x2))

#model
model = keras.Model(
    inputs=[sans_input],
    outputs=output_layers,
)


2023-03-23 22:00:51.447660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def fn_regression_loss(y_true, y_pred):
    rank = tf.rank(y_true)
    if rank == 2:
        pattern = tf.zeros_like(y_true[0])
        shape = y_true.shape[1]
        mask = tf.math.equal(y_true, pattern)
        split = tf.split(mask, num_or_size_splits=shape, axis=1)
    else:
        # unsure this function will ever get a one-dimensional vector
        pattern = tf.zeros_like(y_true)
        shape = y_true.shape[0]
        mask = tf.math.equal(y_true, pattern)
        split = tf.split(mask, num_or_size_splits=shape, axis=0)

    y_mult = split[0]
    for i in range(1, shape):
        y_mult = tf.math.logical_and(y_mult, split[i])
    y_mult = tf.where(y_mult, 0., 1.)

    mse = tf.reduce_mean((tf.squeeze(y_pred)-tf.squeeze(y_true))**2, axis=1)
    mse_masked = tf.math.multiply(tf.squeeze(mse), tf.squeeze(y_mult))
    result = tf.reduce_mean(mse_masked)

    return result

opt = keras.optimizers.Adam(learning_rate=0.0005)
losslist = [fn_regression_loss] * len(sans_models)
losslist.append(keras.losses.CategoricalCrossentropy)

model.compile(optimizer=opt, loss=losslist)
print(model.summary())